In [4]:
from dotenv import load_dotenv
load_dotenv(override=True)

import os
import json

from langchain.vectorstores import FAISS
# FAISS (Facebook AI Similarity Search) to biblioteka opracowana przez Facebook AI Research do wydajnego wyszukiwania podobieństw i grupowania gęstych wektorów.

from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.docstore.document import Document
from langchain.schema import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

In [5]:
VECTOR_STORE_PATH = "memory.index"
MEMORY_PATH = "memory.md"

def get_vector_store():
    # stworzenie memory.index jeśli katalog nie istnieje
    if os.path.exists(VECTOR_STORE_PATH):
        return FAISS.load_local(VECTOR_STORE_PATH, embeddings=OpenAIEmbeddings(), allow_dangerous_deserialization=True)
    
    # załadowanie pliku do dokumentu
    loader = TextLoader(MEMORY_PATH)
    memory = loader.load()[0]
    documents = [Document(page_content=content) for content in memory.page_content.split("\n\n")]

    # konwersja tekstu na embeddingi, stworzenie indeksów, stworzenie i zapisanie obiektu FAISS
    store = FAISS.from_documents(documents, OpenAIEmbeddings())
    store.save_local(VECTOR_STORE_PATH)
    return store

In [6]:
query = "Write a summary of the games by AI_Devs."

# wyszukanie powiązanych dokumentów przez similarity search
vector_store = get_vector_store()
context = vector_store.similarity_search_with_score(query, k=2)
context_document, context_score = context[0] # wzięcie top1 dokumentu i jego wyniku


chat = ChatOpenAI()
person = chat.invoke([
    SystemMessage(f"""
        Assign the task provided by the user to the person who is most likely to complete it based on the context and nothing else.
        Return the lowercase name or "general" if you can't find a match.###{context_document.page_content if context else ''}###
    """),
    HumanMessage(query),
])

print("Notify:", person.content)

Notify: jakub
